In [2]:
using Pkg
### use the Project and Manifest tomls to construct same environment, then activate:
Pkg.activate(".")

using ACE1pack, ACE1, IPFitting
using Statistics
using Random
using LinearAlgebra
using Distributions
using Plots
using StatsPlots

### set plotting defaults
default(fmt = :png,size=(800,800),ms=6,xtickfontsize=10,ytickfontsize=10,xguidefontsize=16,yguidefontsize=16,legendfontsize=10,titlefontsize=16)

using LaTeXStrings
using ColorSchemes
using DataFrames
using CSV

### Use own modules
using IB_BayesianLinearRegression
using UQPotentials
using ConformalPredictions

### Use code snippets
include("./code/ACE_fs.jl")
using .ACE_fs

include("./code/QoIEvaluators.jl")
using .QoIEvaluators

In [ ]:
datapath = "./data/gp_iter6_sparse9k.xml.xyz"
isolated_atom_energy = IPFitting.Data.read_xyz(datapath, energy_key="dft_energy", force_key="dft_force", 
                            virial_key="dft_virial", include=["isolated_atom";], verbose=false)[1].D["E"][1];

In [ ]:
data = IPFitting.Data.read_xyz(datapath, energy_key="dft_energy", force_key="dft_force", 
                    virial_key="dft_virial", include=["dia","vacancy","divacancy","interstitial"], 
                    verbose=false);

In [ ]:
### DFT reference results

dft_bulk_modulus = 88.596696666666602

dft_c11=153.28990999999988
dft_c12=56.25008999999995
dft_c44=72.176929999999999;

dft_vfe = 3.6732338007932412;

dft_migration_path = [3.62673612056642, 3.6525185844820953, 3.722158915757973, 3.825509338568736, 
    3.9117284704134363, 3.925445864948415, 3.8606456924135273, 3.761658791161608, 3.6808647102116083, 
    3.6386444156723883, 3.6267294104800385];

In [ ]:
### Figure 1



# savefig(plt1, "./figures/Figure1.png")
display(plt1)

In [ ]:
### Figure 2

### read in data
deg_site_list = collect(3:24)

k = collect(25:25:350)
dfs2 = DataFrame.(CSV.File.(["./outputs/bulk_modulus/dia$(i).csv" for i in k]));

### numbers corresponding to k, max poly degree / deg site list values for plotting
num_coeffs = [9.0, 13.0, 19.0, 28.0, 40.0, 57.0, 81.0, 113.0, 156.0, 214.0, 289.0, 386.0, 512.0, 671.0, 871.0, 
    1122.0, 1432.0, 1812.0, 2278.0, 2841.0, 3518.0, 4330.0];
num_data = [2029.0, 3692.0, 6123.0, 10192.0, 13151.0, 17490.0, 20101.0, 21680.0, 24315.0, 27706.0, 30533.0,
    33462.0, 36127.0, 38870.0];

    x1_ = collect(1:14)
    x2_ = collect(1:22)

### track if prediction(s) agree with DFT, save for plotting
a_ = []
b_ = []
for x1 in x1_
    for x2 in x2_        
        set = [mean(eval(Meta.parse(dfs2[x1][x2,"QoI_list"]))) - std(eval(Meta.parse(dfs2[x1][x2,"QoI_list"]))) * dfs2[x1][x2,"q̂_list"],mean(eval(Meta.parse(dfs2[x1][x2,"QoI_list"]))) + std(eval(Meta.parse(dfs2[x1][x2,"QoI_list"]))) * dfs2[x1][x2,"q̂_list"]]
        if (set[1] < dft_bulk_modulus < set[2]) == false
            push!(a_,x1)
            push!(b_,x2)
        end
    end
end

### get st devs and errors for bulk modulus
B_stds = Matrix{Float64}(undef,length(deg_site_list),length(k))
B_errs = Matrix{Float64}(undef,length(deg_site_list),length(k))
for (count,df) in enumerate(dfs2)
    a = eval.(Meta.parse.(df[1:22,"QoI_list"]))
    for i in 1:length(a)
        b = a[i][findall(x->x>0.0,a[i])]
        B_stds[i,count] = std(b)
        B_errs[i,count] = std(b) * df.q̂_list[i]
    end
end


clims = (0.0009450134903232765, 99.89156612653741)
num_contour_levels = 50

### do contour plotting of conformal errors
plt2 = contour(k,deg_site_list,B_errs,colour=:turbo,clabels=false,cb=:left,cbar=false,fill=true,yticks=false,
    clims=clims,levels=num_contour_levels)
xlabel!(plt2,"# Configs")
ylabel!(plt2,"Max Poly Degree")
yticks!(plt2,deg_site_list,string.(Int.(deg_site_list)))
scatter!(twinx(),deg_site_list,repeat([NaN],length(num_coeffs)),label="",yticks=(deg_site_list,string.(Int.(num_coeffs))),ylabel="# Coefficients")
scatter!(twiny(),k,repeat([NaN],length(k)),label="",xticks=(k,string.(Int.(num_data))),xlabel="# Data points")
xlims!((k[1],k[end]))
ylims!((deg_site_list[1],deg_site_list[end]))

### mark points which do not agree with DFT reference
for i in 1:length(a_)
    println(a_[i],b_[i])
    scatter!(plt2,[k[a_[i]]],[deg_site_list[b_[i]]],marker=:x,markercolour=:white,label="")
end

# savefig(plt2, "./figures/Figure2.png")
display(plt2)

In [ ]:
### Figure 3

### get q̂ values
qhat_matrix = Matrix{Float64}(undef,length(deg_site_list),length(k))
for (count,df) in enumerate(dfs2)
    qhat_matrix[:,count] = df[1:22,"q̂_list"]
end

# qhat_matrix
qhat_plot = contour(k,deg_site_list,qhat_matrix,colour=:turbo,clabels=false,cbar=true,fill=true,levels=num_contour_levels,
    colorbar_title = L"\hat{q}",colorbar_titlefontsize=16)
yticks!(deg_site_list,string.(Int.(deg_site_list)))
xlabel!("Training Configurations")
ylabel!(L"p_{\mathrm{max}}")

# stds_matrix
ensemble_sigma_plot = contour(k,deg_site_list,B_stds,colour=:turbo,clabels=false,cbar=true,fill=true,levels=num_contour_levels,
    colorbar_title=L"\mathrm{Ensemble \,\,} \sigma",colorbar_titlefontsize=16)
yticks!(deg_site_list,string.(Int.(deg_site_list)))
xlabel!("Training Configurations")
ylabel!(L"p_{\mathrm{max}}")

plt3 = plot(qhat_plot,ensemble_sigma_plot,size=(1600,800))

# savefig(plt3, "./figures/Figure3.png")
display(plt3)

In [ ]:
### Figure 4

### read in data
### <warning> since this was generated piece-wise, read-in and plotting looks slightly ugly...</warning>

ks = [25,50,75,100,125,150,175,200,225,250,275,300,325]

df_strings = [string("./outputs/elastic_constants/dia$(k).csv") for k in ks];
dfs4 = DataFrame.(CSV.File.(df_strings));

df_strings_ = [string("./outputs/elastic_constants/dia$(k)_part2.csv") for k in ks];
dfs4_ = DataFrame.(CSV.File.(df_strings_));

### setup (sub-)plots
p1 = plot()
hline!(p1,[dft_c11],ls=:dash,label="",ylabel=L"c_{11} \, \mathrm{ (GPa)}",xlabel="",lw=1.5)
p2 = plot()
hline!(p2,[dft_c12],ls=:dash,label="",ylabel=L"c_{12} \, \mathrm{ (GPa)}",xlabel="",lw=1.5)
p3 = plot()
hline!(p3,[dft_c44],ls=:dash,label="",ylabel=L"c_{44} \, \mathrm{ (GPa)}",xlabel="",lw=1.5)
p4 = plot()
hline!(p4,[dft_c11],ls=:dash,label="",ylabel=L"c_{11} \, \mathrm{ (GPa)}",xlabel="",lw=1.5)
p5 = plot()
hline!(p5,[dft_c12],ls=:dash,label="",ylabel=L"c_{12} \, \mathrm{ (GPa)}",xlabel="Training configurations",lw=1.5)
p6 = plot()
hline!(p6,[dft_c44],ls=:dash,label="",ylabel=L"c_{44} \, \mathrm{ (GPa)}",xlabel="",lw=1.5)
p7 = plot();
hline!(p7,[dft_c11],ls=:dash,label="",ylabel=L"c_{11} \, \mathrm{ (GPa)}",xlabel="",lw=1.5);
p8 = plot();
hline!(p8,[dft_c12],ls=:dash,label="DFT",ylabel=L"c_{12} \, \mathrm{ (GPa)}",xlabel="",lw=1.5);
title!(p8,L"p_{\mathrm{max}} = 12, 214 \,\, \mathrm{ parameters}")
p9 = plot();
hline!(p9,[dft_c44],ls=:dash,label="",ylabel=L"c_{44} \, \mathrm{ (GPa)}",xlabel="",lw=1.5);

### enumerate throught relevant dataframes/plots

low = [100000000.0,100000000.0,100000000.0,100000000.0,100000000.0,100000000.0]
up = [-100000000.0,-100000000.0,-100000000.0,-100000000.0,-100000000.0,-100000000.0]

for x in [1,2]
    
    if x == 1
        ps = [p1,p2,p3]
        title!(ps[2],L"p_{\mathrm{max}} = 4, 13 \,\, \mathrm{ parameters}")
    else
        ps = [p4,p5,p6]
        title!(ps[2],L"p_{\mathrm{max}} = 19, 1432 \,\, \mathrm{ parameters}")
    end
    
    for (c,df) in enumerate(dfs4)

        c11,c12,c44 = UQPotentials.unpack_vecvec_of_3tuples(eval.(Meta.parse.(df[!,"QoI_list"])));

        c11,c12,c44 = c11[x],c12[x],c44[x]

        q̂ = df.q̂_list[x]

        for (count,QoI) in enumerate([c11,c12,c44])
            mu = mean(QoI)
            sigma = std(QoI)

            l,u = conformalpredictionset(mu,sigma,q̂)
            if l < low[count]
                low[count] = l
            end
            if u > up[count]
                up[count] = u
            end

            mult = 10
            l_round = floor.(low./mult).*mult
            u_round = ceil.(up./mult).*mult
            n_ticks = (u_round - l_round)/mult .+1
            max_n_ticks,pos = findmax(n_ticks)

            for i in 1:3
                if isodd(abs(n_ticks[i] - max_n_ticks))
                    l_round[i] -= div(Int(abs(n_ticks[i] - max_n_ticks)),2)*mult
                    u_round[i] += rem(Int(abs(n_ticks[i] - max_n_ticks)),2)*mult
                else
                    l_round[i] -= div(Int(abs(n_ticks[i] - max_n_ticks)),2)*mult
                    u_round[i] += div(Int(abs(n_ticks[i] - max_n_ticks)),2)*mult 
                end
            end

            scatter!(ps[count],[ks[c]],[mu],yerr=q̂*sigma,label="",markercolour=:black,marker=:x,ms=6,
                yticks=Int(max_n_ticks))
            ylims!((l_round[count],u_round[count]))
        end
    end
end

x=1
    
for (c,df) in enumerate(dfs4_)

    c11,c12,c44 = UQPotentials.unpack_vecvec_of_3tuples(eval.(Meta.parse.(df[!,"QoI_list"])));

    c11,c12,c44 = c11[x],c12[x],c44[x]

    q̂ = df.q̂_list[x]

    for (count,QoI) in enumerate([c11,c12,c44])
        mu = mean(QoI)
        sigma = std(QoI)

        l,u = conformalpredictionset(mu,sigma,q̂)
        if l < low[count]
            low[count] = l
        end

        if u > up[count]
            up[count] = u
        end

        mult = 10

        l_round = floor.(low./mult).*mult
        u_round = ceil.(up./mult).*mult

        n_ticks = (u_round - l_round)/mult .+1

        max_n_ticks,pos = findmax(n_ticks)

        for i in 1:3

            if isodd(abs(n_ticks[i] - max_n_ticks))

                l_round[i] -= div(Int(abs(n_ticks[i] - max_n_ticks)),2)*mult
                u_round[i] += rem(Int(abs(n_ticks[i] - max_n_ticks)),2)*mult
            else

                l_round[i] -= div(Int(abs(n_ticks[i] - max_n_ticks)),2)*mult
                u_round[i] += div(Int(abs(n_ticks[i] - max_n_ticks)),2)*mult

            end

        end

        scatter!(ps[count],[ks[c]],[mu],yerr=q̂*sigma,label="",markercolour=:black,marker=:x,ms=6,
            yticks=Int(max_n_ticks))
        ylims!((l_round[count],u_round[count]))
    end

end

### collect all subplots
ps = [p1,p2,p3,p7,p8,p9,p4,p5,p6]

### plot!
plt4 = plot(ps...,layout=(3,3),size=(1000,1000));

# savefig(plt4, "./figures/Figure4.png")
display(plt4)

In [ ]:
### Figure 5

q̂s = [i.q̂_list for i in dfs4]
q̂s = vecvec_to_matrix(q̂s)
cs = [UQPotentials.unpack_vecvec_of_3tuples(eval.(Meta.parse.(i[!,"QoI_list"]))) for i in dfs4]
q̂s_ = [i.q̂_list for i in dfs4_]
q̂s_ = vecvec_to_matrix(q̂s_)
cs_ = [UQPotentials.unpack_vecvec_of_3tuples(eval.(Meta.parse.(i[!,"QoI_list"]))) for i in dfs4_]

c11sigmas = []
c12sigmas = []
c44sigmas = []
c11sigmas_ = []
c12sigmas_ = []
c44sigmas_ = []
for i in 1:length(cs)
    push!(c11sigmas,std.(cs[i][1]))
    push!(c12sigmas,std.(cs[i][2]))
    push!(c44sigmas,std.(cs[i][3]))
    push!(c11sigmas_,std.(cs_[i][1]))
    push!(c12sigmas_,std.(cs_[i][2]))
    push!(c44sigmas_,std.(cs_[i][3]))
end
c11sigmas = vecvec_to_matrix(c11sigmas)
c12sigmas = vecvec_to_matrix(c12sigmas)
c44sigmas = vecvec_to_matrix(c44sigmas)
c11sigmas_ = vecvec_to_matrix(c11sigmas_)
c12sigmas_ = vecvec_to_matrix(c12sigmas_)
c44sigmas_ = vecvec_to_matrix(c44sigmas_)

p1 = plot()
xlabel!("")
ylabel!(L"C_{ij} \,\, \mathrm{ensemble \,\, \sigma} \,\,\, \mathrm{ (GPa)}")
title!(L"p_{\mathrm{max}} = 4")
plot!(p1,ks,c11sigmas[:,1],label=L"c_{11}",legend=:right)
plot!(p1,ks,c12sigmas[:,1],label=L"c_{12}")
plot!(p1,ks,c44sigmas[:,1],label=L"c_{44}")
plot!(twinx(),ks,q̂s[:,1],colour=:black,label="")

p2 = plot()
xlabel!("")
title!(L"p_{\mathrm{max}} = 19")
plot!(p2,ks,c11sigmas[:,2],label="",legend=:topleft)
plot!(p2,ks,c12sigmas[:,2],label="")
plot!(p2,ks,c44sigmas[:,2],label="")
plot!(twinx(),ks,q̂s[:,2],colour=:black,label="q̂",ylabel=L"\hat{q}",legend=:right)

p3 = plot()
xlabel!("Training configurations")
title!(L"p_{\mathrm{max}} = 12")
plot!(p3,ks,c11sigmas_[:,1],label="",legend=:topleft)
plot!(p3,ks,c12sigmas_[:,1],label="")
plot!(p3,ks,c44sigmas_[:,1],label="")
plot!(twinx(),ks,q̂s_[:,1],colour=:black,label="")

plt5 = plot(p1,p3,p2,layout=(1,3),size=(1000,400));

# savefig(plt5, "./figures/Figure5.png")
display(plt5)

In [ ]:
### Figure 6

### list of max poly degrees (see csv's)
deg_site_list = collect(3:26);

### read in data
df6 = DataFrame(CSV.File("./outputs/vacancy_formation_energy/dia_vacancy_divacancy500.csv"));
df6_ = DataFrame(CSV.File("./outputs/vacancy_formation_energy/dia_vacancy_divacancy500_part2.csv"));
### combine both dataframes
df6 = vcat(df6,df6_);

### extract list(s) of vfe data from df6
### so vfe_data is vector of vectors.
vfe_data = eval.(Meta.parse.(df6[!,"QoI_list"]))

### Figure 6 plot
plt6 = plot(deg_site_list, mean.(vfe_data), yerror=df6.q̂_list.*std.(zxc), label="",
    xticks=deg_site_list, legend=:right, xlabel=L"p_{\mathrm{max}}", ylabel=L"E_{\mathrm{vac}} \,\, \mathrm{(eV)}",
    seriestype=:scatter, marker=:x, markercolour=:black);
title!(plt6, L"N = 500 \,\, \mathrm{Training \,\, configurations}");

### add DFT reference value
hline!(plt6,[dft_vfe], ls=:dash, label="DFT", lw=1.5, colour=:steelblue2);

### add inset of last few points
### last 12 points
n_ = 11
plot!(plt6, deg_site_list[end-n_:end], mean.(zxc)[end-n_:end], yerror=df6.q̂_list[end-n_:end].*std.(zxc)[end-n_:end], label="",
    xticks=deg_site_list[end-n_:end], inset = (1, bbox(0.05, 0.1, 0.7, 0.35, :bottom, :right)), subplot=2,
    seriestype=:scatter, marker=:x, markercolour=:black);

### add DFT reference value to subplot
hline!(plt6, [dft_vfe], ls=:dash, label="", lw=1.5, subplot=2, colour=:steelblue2);

# savefig(plt6, "./figures/Figure6.png")
display(plt6)

In [ ]:
### Figure 7
plt7 = plot(xlabel=L"p_{\mathrm{max}}", ylabel=L"E_{\mathrm{vac}} \,\, \mathrm{ensemble \,\, \sigma} \,\,\, \mathrm{ (eV)}");
title!(L"N = 500 \,\, \mathrm{Training \,\, configurations}",titlefontsize=16);

plot!(plt7, deg_site_list, std.(vfe_data), label="", legend=(0.87,0.87), xticks=(deg_site_list));
plot!(twinx(), deg_site_list, df6.q̂_list, colour=:black, label="", ylabel="q̂ value");

# savefig(plt7, "./figures/Figure7.png")
display(plt7)

In [ ]:
### Figure 8

function vacancy_migration_plot(fit_positions, fit_Es, q̂, dft_vfe, dft_migration_path, gap_migration_path)

    p1 = plot(xlabel="Reaction coordinate", ylabel="Energy (eV)", title = L"p_{\mathrm{max}} = 20, \rm{Bulk, vacancies, divacancies}")
      
    ### plot DFT path
    dft_pos = LinRange(0.0,1.0,length(dft_migration_path))
    plot!(p1,dft_pos,dft_migration_path,label="DFT")
    
    ### plot GAP path
    plot!(p1,dft_pos,gap_migration_path,label="GAP2018",)
    
    ### mean and stdev of energy across path
    mean_fit_Es = mean(fit_Es)
    std_fit_Es = std(fit_Es)
        
    ### normalise fit positions between 0 and 1
    fit_positions = [fit_positions[i] ./ maximum(fit_positions[i]) for i in 1:length(fit_positions)]
    
    ### plot samples, only label one
    for i in 1:length(fit_positions)
        if i == 1
            plot!(p1,fit_positions[i],fit_Es[i],colour=:blue,linealpha=0.1,label="Samples")
        else
            plot!(p1,fit_positions[i],fit_Es[i],colour=:blue,label="",linealpha=0.1)
        end
    end

    ### plot conformal uncertainty bound and mean result
    plot!(p1,mean(fit_positions),mean_fit_Es,ribbon=std_fit_Es.*q̂,fillalpha=0.2,label="ACE prediction",ls=:dash)
    
    return p1

end

### read in data
df8 = DataFrame(CSV.File("./outputs/vacancy_migration/dia_vacancy_divacancy500_degsite_20.csv"));
### get vacancy migration output
vm_output = eval.(Meta.parse.(df8.QoI_list[1]));
### ...and unpack it
E_barriers, image_positions, image_Es, fit_positions, fit_Es = QoIEvaluators.unpackvacancymigrationoutput(vm_output);

### GAP 2018 vacancy migration path of 11 images for comparison
gap_migration_path = [3.6259608502259653, 3.6460762463193532, 3.6887600503469002, 3.742852516588755, 3.826427906509707, 3.8438765685532417, 
                        3.7791041360778763, 3.7145744405952428, 3.671911670360714, 3.6361868756048352, 3.6259628126226744]

### and do plotting using function above
plt8 = vacancy_migration_plot(fit_positions, fit_Es, df8.q̂_list[1], dft_vfe, dft_migration_path, gap_migration_path);

# savefig(plt8, "./figures/Figure8.png")
display(plt8)